#### Импорт необходимых библиотек

In [119]:
import json
import os
import jiwer as jiwer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from vosk import Model, KaldiRecognizer
import wave
import numpy as np
from pydub import AudioSegment
from sklearn.feature_extraction.text import TfidfVectorizer
from fuzzywuzzy import process
from transformers import BertTokenizer, BertModel
import optuna
from gensim.models import Word2Vec
import librosa
import soundfile as sf
import random
from sklearn.metrics import f1_score, precision_score, recall_score
from scipy import signal

#### Константы

In [120]:
# Константы
DATA_DIR = '../data/train/'
DATA_DIR_FILES = [
    # 'hr_bot_clear',
    # 'hr_bot_noise',
    'hr_bot_synt'
]
ANNOTATION_DIR = '../data/train/annotation/'
ANNOTATION_FILES = [
    # 'hr_bot_clear.json',
    # 'hr_bot_noise.json',
    'hr_bot_synt.json'
]

VAL_DIR = '../data/val/luga/'  # Путь к валидационным данным
ANNOTATION_VAL_FILE = os.path.join(VAL_DIR, 'luga.json')

In [121]:
# Настройка Vosk модели для распознавания речи
MODEL_PATH = "../model/vosk_model"
model = Model(MODEL_PATH)
print("Модель Vosk загружена успешно.")

Модель Vosk загружена успешно.


In [122]:
# Метки команд
_label = {
    0: "отказ",
    1: "отмена",
    2: "подтверждение",
    3: "начать осаживание",
    4: "осадить на (количество) вагон",
    5: "продолжаем осаживание",
    6: "зарядка тормозной магистрали",
    7: "вышел из межвагонного пространства",
    8: "продолжаем роспуск",
    9: "растянуть автосцепки",
    10: "протянуть на (количество) вагон",
    11: "отцепка",
    12: "назад на башмак",
    13: "захожу в межвагонное пространство",
    14: "остановка",
    15: "вперед на башмак",
    16: "сжать автосцепки",
    17: "назад с башмака",
    18: "тише",
    19: "вперед с башмака",
    20: "прекратить зарядку тормозной магистрали",
    21: "тормозить",
    22: "отпустить",
}

#### Объявление классов

In [ ]:
def load_noise_samples(noise_dir):
    """Загрузить и объединить шумовые образцы в один аудиофайл"""
    noise_files = [os.path.join(noise_dir, f) for f in os.listdir(noise_dir) if f.endswith('.wav')]
    noises = []
    for noise_file in noise_files:
        noise, sr = librosa.load(noise_file, sr = 16000)
        noises.append(noise)
    combined_noise = np.concatenate(noises)
    return combined_noise, sr

In [126]:
def reduce_noise(audio_file, noise_dir, output_file, noise_reduction_factor=1):
    """
    Удалить шум поезда из голосового сообщения.

    audio_file: путь к аудиофайлу с голосом
    noise_dir: директория с файлами шума
    output_file: куда сохранить очищенный файл
    noise_reduction_factor: коэффициент подавления шума (от 0 до 1)
    """
    # Загрузить аудиофайл с голосом
    voice, sr = librosa.load(audio_file, sr = 16000 )

    # Загрузить шумы
    noise, sr_noise = load_noise_samples(noise_dir)

   # Убедиться, что длина шума соответствует длине аудио (или короче)
    if len(noise) < len(voice):
        noise = np.pad(noise, (0, len(voice) - len(noise)), mode='wrap')
    else:
        noise = noise[:len(voice)]

    # Преобразование аудио и шума в STFT (Short-Time Fourier Transform)
    voice_stft = librosa.stft(voice)
    noise_stft = librosa.stft(noise)

    # Усреднить шумы по времени
    noise_mean = np.mean(np.abs(noise_stft), axis=1)

    # Вычесть шум из голоса
    voice_clean_stft = voice_stft - noise_reduction_factor * noise_mean[:, np.newaxis]

    # Преобразовать обратно в аудиосигнал
    voice_clean = librosa.istft(voice_clean_stft)

    # Сохранить очищенный аудиофайл
    sf.write(output_file, voice_clean, sr)


reduce_noise(audio_file, noise_dir, output_file)

In [124]:
# Датасет для классификации текста
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

In [125]:
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)  # output_dim соответствует количеству классов

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  # Возвращаем тензор с размером [batch_size, output_dim]

##### Объявление функций

In [127]:
# Функция для обработки аудиофайла
def transcribe_audio(audio_file, dir):
    wf = wave.open(f"{dir}/{audio_file}", "rb")
    rec = KaldiRecognizer(model, wf.getframerate())

    result_text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            result_text += result.get("text", "")

    final_result = json.loads(rec.FinalResult())
    result_text += final_result.get("text", "")

    return result_text

In [128]:
# Функция классификации текста
def classify_text(text, classifier, tokenizer):
    # Преобразование текста в вектор
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = classifier(inputs['input_ids'])
    return outputs

In [129]:
# Аугментация аудио
def augment_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    noise = np.random.randn(len(audio))
    augmented_audio = audio + 0.005 * noise
    augmented_file_path = file_path.replace('.wav', '_augmented.wav')
    sf.write(augmented_file_path, augmented_audio, sr)
    return augmented_file_path

In [130]:
# Загрузка аннотаций
def load_annotations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [131]:
def encode_with_bert(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

In [132]:
# Преобразование MP3 в WAV
def convert_mp3_to_wav(mp3_filepath):
    try:
        if not mp3_filepath.endswith('.mp3'):
            print(f"Файл {mp3_filepath} не является MP3.")
            return None

        wav_filepath = mp3_filepath.replace('.mp3', '.wav')
        audio = AudioSegment.from_file(mp3_filepath)
        audio.export(wav_filepath, format='wav')
        print(f"Файл {mp3_filepath} успешно конвертирован в WAV: {wav_filepath}")
        return wav_filepath
    except Exception as e:
        print(f"Ошибка при конвертации {mp3_filepath} в WAV: {e}")
        return None

In [133]:
def load_dataset():
    audio_files = []
    texts = []
    labels = []

    for annotation_file, data_dir in zip(ANNOTATION_FILES, DATA_DIR_FILES):
        annotation_path = os.path.join(ANNOTATION_DIR, annotation_file)
        print(f"Загрузка аннотаций из: {annotation_path}")
        training_annotations = load_annotations(annotation_path)

        for annotation in training_annotations:
            audio_filepath = os.path.join(DATA_DIR, data_dir, annotation['audio_filepath'])
            if os.path.exists(audio_filepath):
                if audio_filepath.endswith('.mp3'):
                    audio_filepath = convert_mp3_to_wav(audio_filepath)

                # Подавляем шум перед аугментацией
                denoised_file = denoise_audio(audio_filepath)

                augmented_file = augment_audio(denoised_file)
                audio_files.append(augmented_file)

                text = annotation['text']
                label = annotation['label']
                texts.append(text)  # добавляем текст
                labels.append(label)  # добавляем метку

            else:
                print(f"Файл {audio_filepath} не найден.")
                continue

    print("Загрузка датасета завершена.")
    return audio_files, texts, labels

#### Подготовка настроек модели, создание модели

In [134]:
# Вычисление WER
def calculate_wer(reference, hypothesis):
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    S = sum(1 for r, h in zip(reference_words, hypothesis_words) if r != h)
    D = len(reference_words) - len(hypothesis_words) if len(reference_words) > len(hypothesis_words) else 0
    I = len(hypothesis_words) - len(reference_words) if len(hypothesis_words) > len(reference_words) else 0
    N = len(reference_words)

    return (S + D + I) / N if N > 0 else 0

In [135]:
# Вычисление Mq
def calculate_mq(wer, f1_weighted):
    return 0.25 * (1 - wer) + 0.75 * f1_weighted

In [138]:
# Функция objective для Optuna
def objective(trial):
    hidden_dim = trial.suggest_int('hidden_dim', 32, 128)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 4, 16)

    # Загрузка и подготовка данных
    audio_files, texts, labels = load_dataset()
    train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

    # Генерация векторов BERT
    bert_vectors = torch.stack([encode_with_bert(text) for text in train_texts])

    # Преобразование меток в тензор
    train_labels = torch.tensor(train_labels, dtype=torch.long)

    # Создание датасета и загрузчика
    train_dataset = TextDataset(bert_vectors, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Определение количества классов
    num_classes = len(set(labels))

    # Обучение модели
    classifier = TextClassifier(input_dim=bert_vectors.size(1), hidden_dim=hidden_dim, output_dim=num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

    classifier.train()  # Переводим модель в режим обучения
    for epoch in range(3):  # Можно увеличить количество эпох
        for texts_batch, labels_batch in train_loader:
            optimizer.zero_grad()
            outputs = classifier(texts_batch.float())

            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()

    # Оценка модели на тестовом наборе
    classifier.eval()  # Переводим модель в режим оценки
    test_bert_vectors = torch.stack([encode_with_bert(text) for text in test_texts])
    test_dataset = TextDataset(test_bert_vectors, torch.tensor(test_labels, dtype=torch.long))
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for texts_batch, labels_batch in test_loader:
            outputs = classifier(texts_batch.float())
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.numpy())
            all_labels.extend(labels_batch.numpy())

    # Теперь вычисляем WER для каждого примера и усредняем
    total_wer = 0
    for ref, hyp in zip(test_texts, all_preds):
        total_wer += calculate_wer(ref, _label[hyp])  # Используем _label для преобразования предсказания в текст

    average_wer = total_wer / len(test_texts)

    # Вычисление метрик
    f1_weighted = f1_score(all_labels, all_preds, average='weighted')
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    mq = calculate_mq(average_wer, f1_weighted)

    return mq

In [139]:
# Настройка модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [140]:
# Оптимизация с помощью Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2024-10-13 04:03:50,005] A new study created in memory with name: no-name-05e177e0-1fd6-4c5c-8299-0e01511a0add
C:\Users\nikit\AppData\Local\Temp\ipykernel_30368\4250569102.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
[W 2024-10-13 04:03:50,011] Trial 0 failed with parameters: {'hidden_dim': 55, 'learning_rate': 0.00021714775093360997, 'batch_size': 12} because of the following error: PermissionError(13, 'Permission denied').
Traceback (most recent call last):
  File "D:\russian_railways\002_train_operator_console\venv\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\nikit\AppData\Local\Temp\ipykernel_30368\4250569102.py", line 7, in objective
    audio_files, texts, labels = loa

PermissionError: [Errno 13] Permission denied: '../data/train/annotation/extra'

In [ ]:
print("Лучшие параметры:")
print(study.best_params)
print("Лучшее значение функции цели:")
print(study.best_value)